In [1]:
import json
from tqdm import tqdm

In [2]:
big = "memoryalpha"
small = "stexpanded"
top_k = 10

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/_matched_communities_leiden/top_10_pairs/" + small + "-" + big + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
big_communities = []
gold_pairs = []
matched_communities = []

In [3]:
with (open(communities_big_file) as cbf, open(communities_small_file) as csf,
      open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [4]:
matched_communities

[[0, [0, 367, 27, 220, 269, 753, 1203, 816, 1357, 2115]],
 [1, [1037, 1220, 1999, 1905, 1997, 426, 48, 2209, 286, 407]],
 [2, [856, 857, 365, 855, 1992, 366, 2032, 85, 469, 1652]],
 [3, [460, 2049, 491, 450, 477, 315, 1031, 1050, 606, 2093]],
 [4, [912, 2307, 911, 910, 2094, 2306, 913, 481, 620, 56]],
 [5, [3, 755, 572, 701, 742, 685, 811, 29, 716, 722]],
 [6, [910, 911, 912, 481, 1665, 877, 2094, 2307, 913, 909]],
 [7, [910, 911, 912, 2307, 481, 1665, 877, 2306, 913, 2094]],
 [8, [911, 910, 2307, 2306, 912, 481, 877, 57, 2094, 1665]],
 [9, [910, 912, 911, 2307, 481, 97, 1665, 877, 2094, 2306]],
 [10, [97, 1244, 935, 85, 925, 266, 1050, 923, 2032, 469]],
 [11, [469, 1050, 468, 445, 2024, 97, 1054, 69, 444, 88]],
 [12, [2101, 448, 2102, 2091, 506, 445, 426, 1050, 2023, 2024]],
 [13, [1050, 444, 445, 97, 469, 1054, 481, 448, 407, 69]],
 [14, [1037, 469, 445, 468, 69, 1050, 87, 407, 85, 1905]],
 [15, [607, 1665, 97, 85, 1639, 96, 2234, 877, 422, 426]],
 [16, [2234, 422, 1037, 97, 375, 86,

In [5]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [6]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        community_1 = small_communities[matched_community[0]]
        community_2 = set()
        for i in range(0, top_k):
            community_2.update(big_communities[matched_community[1][i]])
        if gold_pair[1] in community_1 and gold_pair[0] in community_2:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:02<00:00, 62.82it/s]

Gold pairs in matched communities top 10: 37.83783783783784%


In [7]:
found_big = 0
found_small = 0
for gold_pair in gold_not_exact:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")
print("Small: " + str(found_small / all_pairs * 100) + "%")

Big: 100.0%
Small: 100.0%
